In [1]:
import numpy as np
import torch

m = n = 28
xgrid = np.linspace(-1, 1, m)
ygrid = np.linspace(1, -1, n)
x0, x1 = np.meshgrid(xgrid, ygrid)
x_coord = np.stack([x0.ravel(), x1.ravel()], 1)
x_coord = torch.from_numpy(x_coord).float()

In [2]:
x_coord.shape

torch.Size([784, 2])

In [31]:
x = x_coord.expand(32, 784, 2)
print(x.size())
print(x[0][0])
print(x[1][0])
print(x[1, 0])
print(x.contiguous().size())

torch.Size([32, 784, 2])
tensor([-1.,  1.])
tensor([-1.,  1.])
tensor([-1.,  1.])
torch.Size([32, 784, 2])


In [34]:
from torch.autograd import Variable

theta = torch.randn((32,))
rot = Variable(theta.data.new(32, 2, 2).zero_())
rot[:, 0, 0] = torch.cos(theta)
rot[:, 0, 1] = torch.sin(theta)
rot[:, 1, 0] = -torch.sin(theta)
rot[:, 1, 1] = torch.cos(theta)

x = torch.bmm(x, rot) ## 让batch里面的每张图片的每个像素坐标旋转对应的\theta度
x.shape

torch.Size([32, 784, 2])

# **Variational Autoencoder with a Normalizing Flow prior** ${\P}$ 

In [3]:
# import pyro
import torch
import torch.nn as nn
import torch.utils.data as data
# import zuko

# from pyro.contrib.zuko import ZukoToPyro
# from pyro.optim import Adam
# from pyro.infer import SVI, Trace_ELBO
from torch import Tensor
from torchvision.datasets import MNIST
from torchvision.transforms.functional import to_tensor, to_pil_image
from tqdm import tqdm

## Data

In [4]:
trainset = MNIST(root="./", download=True, train=True, transform=to_tensor)
trainloader = data.DataLoader(trainset, batch_size=256, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:04<00:00, 2025398.69it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 114080.61it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1137131.65it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1920222.64it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [3]:
x = [trainset[i][0] for i in range(16)]
x = torch.cat(x, dim=-1)

to_pil_image(x)

## Model

In [4]:
class GaussianEncoder(nn.Module):
    def __init__(self, features: int, latent: int):
        super().__init__()

        self.hyper = nn.Sequential(
            nn.Linear(features, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2 * latent),
        )

    def forward(self, x: Tensor):
        phi = self.hyper(x)
        mu, log_sigma = phi.chunk(2, dim=-1)

        return pyro.distributions.Normal(mu, log_sigma.exp()).to_event(1)


class BernoulliDecoder(nn.Module):
    def __init__(self, features: int, latent: int):
        super().__init__()

        self.hyper = nn.Sequential(
            nn.Linear(latent, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, features),
        )

    def forward(self, z: Tensor):
        phi = self.hyper(z)
        rho = torch.sigmoid(phi)

        return pyro.distributions.Bernoulli(rho).to_event(1)

In [23]:
BDe = BernoulliDecoder(256, 32)
x = torch.randn(32, 32)
out = BDe(x)
out.mean.shape

torch.Size([32, 256])

In [5]:
class VAE(nn.Module):
    def __init__(self, features: int, latent: int = 16):
        super().__init__()

        self.encoder = GaussianEncoder(features, latent)
        self.decoder = BernoulliDecoder(features, latent)

        self.prior = zuko.flows.MAF(
            features=latent,
            transforms=3,
            hidden_features=(256, 256),
        )

    def model(self, x: Tensor):
        """
        pyro.sample 定义从概率分布中生成数据或潜变量。
        """
        pyro.module("prior", self.prior)
        pyro.module("decoder", self.decoder)

        with pyro.plate("batch", len(x)):
            z = pyro.sample("z", ZukoToPyro(self.prior()))
            x = pyro.sample("x", self.decoder(z), obs=x)

    def guide(self, x: Tensor):
        """
        pyro.sample 定义近似分布，用来推断 model 中的潜变量。
        """
        pyro.module("encoder", self.encoder)

        with pyro.plate("batch", len(x)):
            z = pyro.sample("z", self.encoder(x))

In [ ]:
vae = VAE(784, 16).to("mps")
vae

In [9]:
pyro.clear_param_store()

svi = SVI(vae.model, vae.guide, Adam({'lr': 1e-3}), loss=Trace_ELBO())

for epoch in (bar := tqdm(range(96))):
    losses = []

    for x, _ in trainloader:
        x = x.round().flatten(-3).to("mps")

        losses.append(svi.step(x))

    losses = torch.tensor(losses)

    bar.set_postfix(loss=losses.sum().item() / len(trainset))

100%|██████████| 96/96 [10:10<00:00,  6.36s/it, loss=62.9]


In [ ]:
z = vae.prior().sample((16,))
x = vae.decoder(z).mean.reshape(-1, 28, 28)

to_pil_image(x.movedim(0, 1).reshape(28, -1))

torch.Size([16, 16])


torch.Size([28, 28])